In [634]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [635]:
xlsx = pd.ExcelFile("DataSet.xlsx")
df = pd.read_excel(xlsx)



In [636]:
#Monthly return sp500
df['sp500ret']= df['SP500Index'].pct_change()

In [637]:
#Monthly return of bond index
df['bondret'] = df['BarclaysUSAggTreasury Index'].pct_change()


In [638]:
#Benchmark Portfolio, shows first 5 monthly returns
df['benchmarkret'] = .6*df['sp500ret'] + .4*df['bondret']
df['benchmarkret'].head()


0         NaN
1    0.008450
2    0.015780
3   -0.018664
4    0.069750
Name: benchmarkret, dtype: float64

In [639]:
#adding 1 to returns for caluclation purposes
df['1+sp500ret']= 1+ df['sp500ret']
df['1+bondret']=1+ df['bondret']
df['1+benchmarkret'] = 1 + df['benchmarkret']


In [640]:
#Total Compound Cumulative Returns 
#sp500CumRet = (df["SP500Index"][len(df.index)-1] / df['SP500Index'][0]) -1
#bondCumRet = (df['BarclaysUSAggTreasury Index'][len(df.index)-1] / df['BarclaysUSAggTreasury Index'][0])
df['sp500CumRet']= df['1+sp500ret'].cumprod()-1
df['bondCumRet'] = df['1+bondret'].cumprod()-1
df['benchmarkCumRet'] = df['1+benchmarkret'].cumprod()-1



In [644]:
#Annualized Returns, 
sp500AnnRet = (df["sp500CumRet"][len(df.index)-1])**(1/27.)-1
bondAnnRet = (df["bondCumRet"][len(df.index)-1])**(1/27.)-1
benchmarkAnnRet = (df["benchmarkCumRet"][len(df.index)-1])**(1/27.)-1
print sp500AnnRet, bondAnnRet, benchmarkAnnRet

0.0931989940909 0.0492857949103 0.0797343968425


In [645]:
#Annual Standard Deviation
sp500SD = np.std(df['sp500ret'])*np.sqrt(12)
bondSD = np.std(df['bondret'])*np.sqrt(12)
benchmarkSD = np.std(df['benchmarkret'])*np.sqrt(12)
print sp500SD, bondSD, benchmarkSD

0.14367923596 0.044111172661 0.086196221143


In [646]:
#annual return of risk free for sharpe ratio calculation
cumRetRF = (df['3m Tbill Index'][len(df.index)-1] /df['3m Tbill Index'][0]) - 1
rfAnnRet = cumRetRF**(1/27.) - 1
rfAnnRet

0.007419355012572959

In [647]:
#sharpe ratio
sp500SharpeRatio = (sp500AnnRet - rfAnnRet)/ sp500SD
bondSharpeRatio = (bondAnnRet - rfAnnRet)/ bondSD
benchmarkSharpeRatio = (benchmarkAnnRet - rfAnnRet )/ benchmarkSD
print sp500SharpeRatio, bondSharpeRatio, benchmarkSharpeRatio

0.597021820899 0.949111922721 0.838958377421


In [648]:
#summary chart for question 2
sp500CumRet = df["sp500CumRet"][len(df.index)-1]
bondCumRet = df["bondCumRet"][len(df.index)-1]
benchmarkCumRet = df["benchmarkCumRet"][len(df.index)-1]
dictSummary = {'Cum Return' : pd.Series([sp500CumRet, bondCumRet, benchmarkCumRet]),
               'Ann Return' : pd.Series([sp500AnnRet, bondAnnRet, benchmarkAnnRet]),
               'Ann Standard Dev' : pd.Series([sp500SD, bondSD, benchmarkSD]),
               'Sharpe Ratio' : pd.Series([sp500SharpeRatio, bondSharpeRatio, benchmarkSharpeRatio])}
                    

summary = pd.DataFrame(dictSummary)
summary1 = summary.set_index([['sp500','barclay','benchmark']])
summary1

,Ann Return,Ann Standard Dev,Cum Return,Sharpe Ratio
sp500,0.093199,0.143679,11.088659,0.597022
barclay,0.049286,0.044111,3.665493,0.949112
benchmark,0.079734,0.086196,7.935189,0.838958


In [649]:
#making new dataframe for questions 3+
df2 = df.iloc[:,0:10]

In [650]:
#Finding returns of the factor
for i in range(0,4):
    df2['factor{}ret'.format(i+1)] = df2['Factor{}'.format(i+1)].pct_change()


In [653]:
#Lagging returns of the factors for prediction purposes, shift factor returns one month down.
for i in range(0,4):
    df2['laggedf{}ret'.format(i+1)] = df2['factor{}ret'.format(i+1)].shift(1)

In [654]:
#10 year data regression

regress1 = smf.ols(formula='sp500ret ~ laggedf1ret', data=df2[2:123]).fit()
regress2 = smf.ols(formula='sp500ret ~ laggedf2ret', data=df2[2:123]).fit()
regress3 = smf.ols(formula='sp500ret ~ laggedf3ret', data=df2[2:123]).fit()
regress4 = smf.ols(formula='sp500ret ~ laggedf4ret', data=df2[2:123]).fit()
print regress3.summary()

                            OLS Regression Results                            
Dep. Variable:               sp500ret   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     12.19
Date:                Sun, 22 Oct 2017   Prob (F-statistic):           0.000675
Time:                        11:20:34   Log-Likelihood:                 226.61
No. Observations:                 121   AIC:                            -449.2
Df Residuals:                     119   BIC:                            -443.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0119      0.004      3.366      

In [655]:
#Calculating regression based off of last 3 years.(factor beta)
for i in range(0,4):
    df2['beta{}'.format(i+1)] = df2['sp500ret'].rolling(window=36).cov(other=df2['laggedf{}ret'.format(i+1)])/ df2['laggedf{}ret'.format(i+1)].rolling(window=36,center=False).var()


In [656]:
#average of sp500 returns from past 3 years
df2['sp500RollingMean'] = df2['sp500ret'].rolling(window=36).mean()

In [657]:
#average of lagged return from past 3 years
for i in range(0,4):
    df2['laggedf{}RollingMean'.format(i+1)] = df2['laggedf{}ret'.format(i+1)].rolling(window=36).mean()


In [658]:
# finding intercept of linear regression
for i in range(0,4):
    df2['f{}intercept'.format(i+1)] = df2['sp500RollingMean'] - df2['beta{}'.format(i+1)]*df2['laggedf{}RollingMean'.format(i+1)]

In [659]:
#Changing values to none because used 10 years worth of regression data to state factor 3 was good.
df2.iloc[:123,18:31] = None 

In [660]:
#Predicted Return( y=mx+b) 
for i in range(0,4):
    df2['f{}PredictRet'.format(i+1)] = df2['f{}intercept'.format(i+1)] + df2['beta{}'.format(i+1)]*df2['laggedf{}ret'.format(i+1)]

In [661]:
#error squared, difference between predicted return and actual return
for i in range(0,4):
    df2['f{}errorSquare'.format(i+1)] = (df2['f{}PredictRet'.format(i+1)]- df2['sp500ret'])**2

In [662]:
#Mean Squared Error, lowest MSE is factor chosen.
# using n-k-1 for denominator of MSE

df2['n-k-1'] = df2.index - 3

print df2['f1errorSquare'].mean(), df2['f2errorSquare'].mean(), df2['f3errorSquare'].mean(), df2['f4errorSquare'].mean()

#Factor 3 is best.

0.00165172017089 0.00173324748022 0.00151698914005 0.00161813324925


In [664]:
#Rebalance formula

df2['equityWeight']=.6


for i in range(123,len(df.index)):
    rebalanceFormula = 1.5 - (1/(1+np.exp(df2['f3PredictRet'][i])))

    if df2['f3PredictRet'][i] >= 0:
        if df2['equityWeight'][i-1]* rebalanceFormula >= .7:
            df2['equityWeight'][i] = .7
        else:
            df2['equityWeight'][i] = df2['equityWeight'][i-1]* rebalanceFormula
    else:
        if df2['equityWeight'][i-1]* rebalanceFormula <= .5:
            df2['equityWeight'][i] = .5
        else:
            df2['equityWeight'][i] = df2['equityWeight'][i-1]*rebalanceFormula


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [665]:
#calculating bondweight of portfolio
df2['bondWeight'] = 1 - df2['equityWeight']


In [666]:
#Setting first couple of rows of equity/bond weights as None,  trading strategy didn't start yet
df2['equityWeight'][:123] = None
df2['bondWeight'][:123] = None


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [667]:
# portfolio return
df2['portfolioRet'] = df2['bondWeight']*df2['bondret'] + df2['equityWeight']*df2['sp500ret']

In [668]:
#alpha
df2['alpha'] = df2['portfolioRet'] - df['benchmarkret']


In [669]:
#cumulative alpha
cumAlpha = (1 + df2['alpha']).prod() - 1
cumAlpha

0.057202690852513394

In [670]:
#annualized alpha
n = 201./12 # #of periods
annualAlpha = (1+ cumAlpha)**(1/n) - 1
annualAlpha


0.0033265026007101017

In [671]:
#tracking error
#df2['alpha'].std()#, also tracking error 
numMonths = 201
trackingError = ((df2['alpha']**2).sum()/numMonths)**(1/2.)
trackingError
            

0.002328710364068098

In [672]:
#Information Ratio
infoRatio = annualAlpha/ trackingError
infoRatio

1.4284741683800164

In [673]:
#random initial portfolio value
df2['portfolioValue'] = 1000000

In [674]:
for i in range(123, len(df2.index)):
    df2['portfolioValue'][i] = df2['portfolioValue'][i-1]* (1+df2['portfolioRet'][i])

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [675]:
# maxdrawdown = (trough - peak)/peak, max loss could possibly earn if invested in this portfolio
#cumulative portfolio return, maximum of cumulative portfolio return
df2['maxPortfolioValue'] = df2['portfolioValue'].cummax() #max value of portfolio up to that date , peak
df2['maxDrawdown'] = (df2['portfolioValue']) / (df2['maxPortfolioValue']) - 1 #maximum value potentially could have lost up to that date
df2['maxDrawdown'].min() 

-0.32766516624774733

In [676]:
#Bonus questions, trading costs
tradingCost = .005*2 #have to trade bond index and stock index
df2['portRetTradingCost'] = None
df2['equityWeight'][122] = .6 # to make loop work
for i in range(123, len(df.index)):
    if df2['equityWeight'][i-1] != df2['equityWeight'][i]:
        df2['portRetTradingCost'][i] = df2['portfolioRet'][i] - tradingCost
    else:
        df2['portRetTradingCost'][i] = df2['portfolioRet'][i]


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [677]:
#check if strategy still returns alpha with previous code
df2['alphaTC'] = df2['portRetTradingCost'] - df['benchmarkret']


In [678]:
#cumulative alpha with trading cost
cumAlphaTC = (1 + df2['alphaTC']).prod() - 1
cumAlphaTC

-0.81597614162514764

In [680]:
#annualized alpha with trading cost
n = 201./12 #of periods 
annualAlphaTC = (1+ cumAlphaTC)**(1/n) - 1
annualAlphaTC

-0.096117686571582484

In [612]:
#question 4

summary2 = pd.DataFrame({'Portfolio Performance' : [cumAlpha,annualAlpha,trackingError,infoRatio,df2['maxDrawdown'].min()]})
summary2.set_index([['Cum Alpha', 'Ann Alpha', 'Tracking Error', 'Info Ratio', 'Max Drawdown']])

,Portfolio Performance
Cum Alpha,0.057203
Ann Alpha,0.003327
Tracking Error,0.002329
Info Ratio,1.428474
Max Drawdown,-0.327665


In [633]:
writer = pd.ExcelWriter('Wellington.xlsx')
df.to_excel(writer,'Sheet1')
df2.to_excel(writer,'Sheet2')
writer.save()